In [1]:
!pip install crewai==0.22.5

In [2]:
!pip install crewai-tools==0.0.15

  Using cached langchain_openai-0.1.13-py3-none-any.whl (45 kB)
  Using cached tiktoken-0.7.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_openai-0.1.12-py3-none-any.whl (43 kB)
  Using cached langchain_openai-0.1.11-py3-none-any.whl (40 kB)
  Using cached langchain_openai-0.1.10-py3-none-any.whl (40 kB)
  Using cached langchain_openai-0.1.9-py3-none-any.whl (40 kB)
  Using cached langchain_openai-0.1.8-py3-none-any.whl (38 kB)
  Using cached langchain_openai-0.1.7-py3-none-any.whl (34 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.5.2
    Uninstalling tiktoken-0.5.2:
      Successfully uninstalled tiktoken-0.5.2
  Attempting uninstall: typer
    Found existing installation: t

In [3]:
!pip install pydantic==2.6.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.2/395.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 43.3 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.18.4
    Uninstalling pydantic_core-2.18.4:
      Successfully uninstalled pydantic_core-2.18.4
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.7.4
    Uninstalling pydantic-2.7.4:
      Successfully uninstalled pydantic-2.7.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
crewai 0.22.5 requires langchain-openai<0.0.6,>=0.0.5, but you have langchain-openai 0.1.7 which is incompatible.
instructor 0.5.2 requires typer<0.10.0,>=0.9.0, but you have typer 0.12.3 which is incompatible.


In [4]:
!pip install langchain-groq==0.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 2.1 MB/s eta 0:00:00


In [1]:
GROQ_API_KEY = "GROQ_API_KEY"

#Setting up the agents

In [6]:
from crewai import Agent
from langchain_groq import ChatGroq

In [22]:
class EmailPersonalizationAgents():
  def __init__(self):
    self.llm = ChatGroq(
        api_key = GROQ_API_KEY,
        model = "Mixtral-8x7B-32768"
    )
  def personalize_email_agent(self):
      return Agent(
          role = "Email Personalizer",
          goal = f"""
          Personalize template emails for recipients using their information.
          Given a template email and recipient information (name, email, bio, last conversation),
          personalize the email by incorporating the recipient's details, into the email, while maintaing, core message
          and structure of the original email.
          This involves updating the introduction, body and closing of the email to make it more personal
          and engaging for each recipient.
          """,
          backstory = """
          As an email personalizer, you are responsible for customizing template emails for
          individual recipients based on their information and previous interaction.
          """,
          verbose = True,
          llm = self.llm,
          max_iter = 2,

      )

  def ghostwriter_agent(self):
        return Agent(
            role = "Ghostwriter",
            goal = f"""
            Revise draft emails to adopt the Ghostwriter's writing style.
            Use an informal, engaging, and slightly sales-oriented tone, mirroring
            the Ghostwriter's final email communication style.
            """,
            backstory = """
            As a ghostwriter, you are responsilbe for revising the draft emails to match the
            Ghostwriter's writing style, focusing on clear, direct communication with a
            friendly and approachable tone.
            """,
            verbose = True,
            llm = self.llm,
            max_iter = 2,
        )

#Setting up the tasks

In [8]:
from crewai import Task
import os

In [9]:
class PersonalizeEmailTask():
  def __init__(self):
    os.makedirs("output",exist_ok=True)

  def personalize_email(self, agent, recipient, email_template):
    return Task(
        description = f"""
        Personalize the template email for recipient using their information.
        - Name : {recipient['first_name']}{recipient['last_name']}
        - Email : {recipient['email']}
        - Bio : {recipient['bio']}
        - Last Conversation : {recipient['last_conversation']}
        Important Information to consider :
        - When personalizing the email, only use one sentence from the bio or last conversation.
        - And make sure, to incorporare it naturally into the email, without going into too much detail.
        - Make sure to keep the updated email roughly the same length as the template email.
        The template email is as follow:
        '''
        {email_template}
        '''
        """,
        agent = agent,
        expected_output = f"Personalized email draft.",
        async_execution = True,
    )
  def ghostwrite_email(self, agent, draft_email, recipient):
    return Task(
       description = f"""
       Revise the draft email to adopt the following writing style.
       Writing style :
       - Use a more informal, engaging, and slightly sales-oriented tone, mirroring ghost writer's final email communication style.
       - This approach prioritizes clear, direct communication while maintaining a friendly and approachable tone.
       - Use straighforward language, including phrases like "Hey [Name]!" to start emails or messages.
       - The tone will be optimistic and encouraging, aiming to build rapport and motivate action, while staying grounded in pratical advice.
       Important Notes :
       - Do not use emojis.
       """,
       agent = agent,
       context = [draft_email],
       expected_output = f"A revised email draft in ghost writer's specified tone and style.",
       output_file = f"output/{recipient['first_name']}_{recipient['last_name']}.txt",
    )

#Bringing everything together

In [10]:
import time
import csv
from crewai import Crew
from langchain_groq import ChatGroq

In [11]:
email_template = """
Hi [Name],

I hope you're doing well. I wanted to touch base regarding our recent conversation about [last_conversation_topic].

We have an active community of professionals who come together for weekly coaching sessions every Tuesday at 6 PM Eastern time. These sessions are a great opportunity to share knowledge, discuss challenges, and collaborate on solutions. Participation is completely free, and we’re thrilled to be nearing the 500-member mark.

With your background in [recipient_bio], your insights would be incredibly valuable to our group. If you have any questions or need advice on your projects, our community is an excellent resource.

Also, if you’re interested in AI and related topics, I think you’d enjoy some of the content on our platform. Make sure to explore our resources and stay up-to-date with the latest discussions.

Looking forward to seeing you in the community!

Best,
Pranav J S
"""

In [12]:
agents = EmailPersonalizationAgents()
email_personalizer = agents.personalize_email_agent()
ghostwriter = agents.ghostwriter_agent()

In [13]:
tasks = PersonalizeEmailTask()
personalize_email_tasks = []
ghostwrite_email_tasks = []

In [14]:
csv_file = 'clients_small.csv'

In [15]:
with open(csv_file, mode='r', newline='') as file:
  csv_reader = csv.DictReader(file)

  for row in csv_reader:
    recipient = {
        'first_name' : row['first_name'],
        'last_name' : row['last_name'],
        'email' : row['email'],
        'bio' : row['bio'],
        'last_conversation' : row['last_conversation']
    }

    personalize_email_task = tasks.personalize_email(
        agent = email_personalizer,
        recipient = recipient,
        email_template = email_template
    )

    ghostwrite_email_task = tasks.ghostwrite_email(
        agent = ghostwriter,
        draft_email = personalize_email_task,
        recipient = recipient
    )

    personalize_email_tasks.append(personalize_email_task)
    ghostwrite_email_tasks.append(ghostwrite_email_task)

#Setting up the crew

In [16]:
crew = Crew(
    agents = [
        email_personalizer,
        ghostwriter
    ],
    tasks = [
        *personalize_email_tasks,
        *ghostwrite_email_tasks
    ],
    max_rpm=29
)

In [17]:
start_time = time.time()

In [18]:
results = crew.kickoff()



> Entering new CrewAgentExecutor chain...
Final Answer:

Hi John,

I hope you're doing well. I wanted to touch base regarding our recent conversation about the upcoming project timeline and requirements.

We have an active community of professionals who come together for weekly coaching sessions every Tuesday at 6 PM Eastern time. These sessions are a great opportunity to share knowledge, discuss challenges, and collaborate on solutions. Participation is completely free, and we’re thrilled to be nearing the 500-member mark.

With your background as a software engineer with 10+ years of experience in backend development, your insights would be incredibly valuable to our group. If you have any questions or need advice on your projects, our community is an excellent resource.

Also, if you’re interested in AI and related topics, I think you’d enjoy some of the content on our platform. Make sure to explore our resources and stay up-to-date with the latest discussions.

Looking forward to

In [19]:
end_time = time.time()

In [20]:
elapsed_time = end_time - start_time

In [21]:
print(f"Crew kickoff took {elapsed_time} seconds.")
print(f"Crew Usage", crew.usage_metrics)

Crew kickoff took 18.84822940826416 seconds.
Crew Usage {'total_tokens': 2230, 'prompt_tokens': 1771, 'completion_tokens': 459, 'successful_requests': 2}
